# clustering calcium signals
https://docs.google.com/document/d/18pjHYOuAd8yLlwQOMr20EXVkvPd2xPTn99PT9L2-_yo/edit#

## The plan
- try rastermap
- 

In [ ]:
![title](bauerLGN.png)


In [ ]:
import tifffile
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import ipywidgets as ipw

## load data

In [ ]:
filepath = Path("../../data/processed")
filepath

In [ ]:
F = np.load()